<a href="https://colab.research.google.com/github/pashanushkarev/PIMonteCarlo/blob/main/PI_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycuda

In [15]:
import numpy as np
import math
import time
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule


In [16]:
kernel = SourceModule("""
__global__ void PiGPU(float *X, float *Y, float *result) 
{
        int index = blockIdx.x * blockDim.x + threadIdx.x; 
        int threadCount = gridDim.x * blockDim.x;
        int n = 1000000;
        int circle = 0;
        for (int i = index; i < n; i += threadCount) {
                if (X[i] * X[i] + Y[i] * Y[i] < 1) {circle++;}
        }
        atomicAdd(result , circle);
}
""")

In [17]:
def PiMonteCarloCPU(n):
    result = 0
    for i in range(n):
        if x[i] ** 2 + y[i] ** 2 < 1: result += 1  
    return result*4/N

In [18]:
N = 1000000
x, y = np.random.random(N), np.random.random(N)
block_size = (256, 1, 1)
grid_size = (int(N / (128 * block_size[0])), 1)
result = np.zeros(1)
PiMonteCarloGPU = kernel.get_function("PiGPU")

In [19]:
start_time_cpu = time.time()
PiMonteCarloCPU(N)
end_time_cpu = time.time() - start_time_cpu
print(f"Time cpu: {end_time_cpu} sec.")

Time cpu: 1.151061773300171 sec.


In [20]:
start_time_gpu = time.time()
PiMonteCarloGPU(drv.In(x), drv.In(y),drv.Out(result), block = block_size, grid = grid_size)
end_time_gpu = time.time() - start_time_gpu
print(f"Time gpu: {end_time_gpu} sec.")

Time gpu: 0.007578134536743164 sec.


In [22]:
SpeedGPU2CPU = end_time_cpu/end_time_gpu
print(f"Ускорение GPU перед CPU: {SpeedGPU2CPU}")

Ускорение GPU перед CPU: 151.89249646059463
